# Описание проекта
Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.  
  
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».  
  
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.  
  
Чтобы оценить результаты опроса, оценки обычно делят на три группы:  
9-10 баллов — «cторонники» (англ. promoters);  
7-8 баллов — «нейтралы» (англ. passives);  
0-6 баллов — «критики» (англ. detractors).  
  
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».  
  
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике.   
  
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 

In [17]:
# Загружаем необходимые библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [26]:
# Пропишем отдельно путь к базе данных
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db_local = r'C:/Users/gawsik/github/Телеком/telecomm_csi.db'
path_to_db = None

In [27]:
# Подключаемся к БД
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_local
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False) # создаём подключение к базе

In [28]:
# Собираем витрину.
query = """
SELECT u.user_id,
	u.lt_day,
	CASE
		WHEN u.lt_day > 365 THEN False
		ELSE True
		END is_new,
	u.age,
	CASE 
           WHEN gender_segment = 1 THEN 'женщина'
           WHEN gender_segment = 0 THEN 'мужчина' -- поправил пол мужчина
           ELSE 'не определен' -- добавил пол "не определен"
    END gender_segment,
	u.os_name,
	u.cpe_type_name,
	l.country,
	l.city,
	substring(age_s.title, 4) AS age_segment,
	substring(t_s.title, 4) AS traffic_segment,
	substring(l_s.title,4) AS lifetime_segment,
	u.nps_score,
	CASE WHEN u.nps_score >= 9 THEN 'cторонники'
	WHEN u.nps_score >= 7 THEN 'нейтралы'
	ELSE 'критики'
	END nps_group
FROM user AS u

JOIN location AS l ON l.location_id = u.location_id
JOIN age_segment AS age_s ON age_s.age_gr_id = u.age_gr_id
JOIN traffic_segment AS t_s ON t_s.tr_gr_id = u.tr_gr_id
JOIN lifetime_segment AS l_s ON l_s.lt_gr_id = u.lt_gr_id
"""

In [29]:
df = pd.read_sql(query, engine) # сохраняем в датафрейм витрину и проверям по условиям.
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,0,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,0,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники


Создаем дашборды.  
Вот [Ссылка на Tableau ](https://public.tableau.com/app/profile/gavrikov.ivan/viz/2_16839110038800/NPS?publish=yes)